This tutorial demonstrates how to use Pixeltable's built-in `vLLM` integration to run local LLMs with high-throughput inference.

<div class="alert alert-block alert-info"><!-- mdx:none -->
<b>If you are running this tutorial in Colab:</b>
vLLM requires a GPU for efficient operation. Click on the <code>Runtime -> Change runtime type</code> menu item at the top, then select the <code>GPU</code> radio button and click on <code>Save</code>.
</div>

### Important notes

- vLLM provides high-throughput inference with techniques like PagedAttention and continuous batching
- Models are loaded from HuggingFace and cached in memory for reuse
- vLLM currently requires a Linux environment with GPU support for best performance
- Consider GPU memory when choosing model sizes

## Set up environment

First, let's install Pixeltable with vLLM support:

In [ ]:
%pip install -qU pixeltable vllm

# For local development, uncomment the next two lines to use the local branch:
# import sys
# sys.path.insert(0, '/path/to/your/pixeltable')


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Create a table for chat completions

Now let's create a table that will contain our inputs and responses.

In [2]:
import pixeltable as pxt
from pixeltable.functions import vllm

pxt.drop_dir('vllm_demo', force=True)
pxt.create_dir('vllm_demo')

t = pxt.create_table('vllm_demo/chat', {'input': pxt.String})

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/pjlb/.pixeltable/pgdata
Created directory 'vllm_demo'.
Created table 'chat'.


/Users/pierre/pixeltable/pixeltable/env.py:494: UserWarning: Progress reporting is disabled because ipywidgets is not installed. To fix this, run: `pip install ipywidgets`
  warnings.warn(


Next, we add a computed column that calls the Pixeltable `chat_completions` UDF, which uses vLLM's high-throughput inference engine under the hood. We specify a HuggingFace model identifier, and vLLM will download and cache the model automatically.

(If this is your first time using Pixeltable, the <a href="https://docs.pixeltable.com/tutorials/tables-and-data-operations">Pixeltable Fundamentals</a> tutorial contains more details about table creation, computed columns, and UDFs.)

For this demo we'll use `Qwen2.5-0.5B-Instruct`, a very small (0.5-billion parameter) model that still produces decent results.

In [3]:
# Add a computed column that uses vLLM for chat completion
# against the input.

messages = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': t.input},
]

t.add_computed_column(
    result=vllm.chat_completions(
        messages,
        model='Qwen/Qwen2.5-0.5B-Instruct',
    )
)

# Extract the output content from the JSON structure returned
# by vLLM.

t.add_computed_column(output=t.result.choices[0].message.content)

Added 0 column values with 0 errors in 0.01 s
Added 0 column values with 0 errors in 0.00 s


No rows affected.

## Test chat completion

Let's try a few queries:

In [4]:
# Test with a few questions
t.insert(
    [
        {'input': 'What is the capital of France?'},
        {'input': 'What are some edible species of fish?'},
        {'input': 'Who are the most prominent classical composers?'},
    ]
)

INFO 02-15 08:32:38 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.
INFO 02-15 08:32:39 [utils.py:263] non-default args: {'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-0.5B-Instruct'}


objc[99479]: Class AVFFrameReceiver is implemented in both /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x10e3ec3a8) and /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x3345283a8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.
objc[99479]: Class AVFAudioReceiver is implemented in both /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x10e3ec3f8) and /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x3345283f8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.


INFO 02-15 08:32:47 [model.py:530] Resolved architecture: Qwen2ForCausalLM
INFO 02-15 08:32:47 [model.py:1545] Using max model len 32768
WARNING 02-15 08:32:47 [cpu.py:157] VLLM_CPU_KVCACHE_SPACE not set. Using 24.0 GiB for KV cache.
INFO 02-15 08:32:47 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=4096.
INFO 02-15 08:32:47 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 02-15 08:32:47 [vllm.py:637] Disabling NCCL for DP synchronization when using async scheduling.
INFO 02-15 08:32:53 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.


objc[99767]: Class AVFFrameReceiver is implemented in both /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x124bc83a8) and /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x3186c43a8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.
objc[99767]: Class AVFAudioReceiver is implemented in both /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x124bc83f8) and /opt/miniconda3/envs/pixeltable/lib/python3.11/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x3186c43f8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.


(EngineCore_DP0 pid=99767) INFO 02-15 08:32:54 [core.py:97] Initializing a V1 LLM engine (v0.14.1) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, enable_return_routed_experts=False, kv_cache_dtype=auto, device_config=cpu, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None, kv_cac

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.34it/s]
(EngineCore_DP0 pid=99767) 


(EngineCore_DP0 pid=99767) INFO 02-15 08:33:33 [default_loader.py:291] Loading weights took 0.75 seconds
(EngineCore_DP0 pid=99767) INFO 02-15 08:33:33 [kv_cache_utils.py:1305] GPU KV cache size: 2,097,152 tokens
(EngineCore_DP0 pid=99767) INFO 02-15 08:33:33 [kv_cache_utils.py:1310] Maximum concurrency for 32,768 tokens per request: 64.00x
(EngineCore_DP0 pid=99767) INFO 02-15 08:33:36 [cpu_model_runner.py:65] Warming up model for the compilation...
(EngineCore_DP0 pid=99767) INFO 02-15 08:33:57 [cpu_model_runner.py:75] Warming up done.
(EngineCore_DP0 pid=99767) INFO 02-15 08:33:57 [core.py:273] init engine (profile, create kv cache, warmup model) took 24.44 seconds
(EngineCore_DP0 pid=99767) INFO 02-15 08:33:59 [vllm.py:630] Asynchronous scheduling is disabled.
(EngineCore_DP0 pid=99767) WARNING 02-15 08:33:59 [vllm.py:672] Inductor compilation was disabled by user settings, optimizations settings that are only active during inductor compilation will be ignored.
(EngineCore_DP0 pid=

3 rows inserted.

In [5]:
t.select(t.input, t.output).collect()

input,output
What is the capital of France?,The capital of France is Paris.
What are some edible species of fish?,"Many species of fish are edible. This includes large fish, such as salmon and"
Who are the most prominent classical composers?,"There are many classical composers who have been influential in the development of classical music,"


## Comparing models

vLLM makes it easy to compare the output of different models. Let's try comparing the output from `Qwen2.5-0.5B` against a somewhat larger model, `Llama-3.2-1B-Instruct`. As always, when we add a new computed column to our table, it's automatically evaluated against the existing table rows.

In [6]:
t.add_computed_column(
    result_l3=vllm.chat_completions(
        messages,
        model='meta-llama/Llama-3.2-1B-Instruct',
    )
)

t.add_computed_column(output_l3=t.result_l3.choices[0].message.content)

t.select(t.input, t.output, t.output_l3).collect()

INFO 02-15 08:34:04 [utils.py:263] non-default args: {'disable_log_stats': True, 'model': 'meta-llama/Llama-3.2-1B-Instruct'}
INFO 02-15 08:34:04 [utils.py:263] non-default args: {'disable_log_stats': True, 'model': 'meta-llama/Llama-3.2-1B-Instruct'}
INFO 02-15 08:34:04 [utils.py:263] non-default args: {'disable_log_stats': True, 'model': 'meta-llama/Llama-3.2-1B-Instruct'}


Error: Error while evaluating computed column 'result_l3':
You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct.
401 Client Error. (Request ID: Root=1-6991f57c-6340aced6d0bbe8c7119ef8b;1cc6949c-ee25-46b7-82e2-8c68490b711e)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

## Using model_kwargs for sampling parameters

vLLM supports fine-grained control over generation through `model_kwargs`. Sampling parameters like `max_tokens`, `temperature`, `top_p`, and `top_k` are passed alongside engine parameters — Pixeltable automatically routes each to the right place. Let's try running with a different system prompt and custom sampling settings.

In [ ]:
messages_teacher = [
    {
        'role': 'system',
        'content': 'You are a patient school teacher. '
        'Explain concepts simply and clearly.',
    },
    {'role': 'user', 'content': t.input},
]

t.add_computed_column(
    result_teacher=vllm.chat_completions(
        messages_teacher,
        model='Qwen/Qwen2.5-0.5B-Instruct',
        model_kwargs={'max_tokens': 256, 'temperature': 0.7, 'top_p': 0.9},
    )
)

t.add_computed_column(
    output_teacher=t.result_teacher.choices[0].message.content
)

t.select(t.input, t.output_teacher).collect()

## Text generation

In addition to chat completions, vLLM also supports direct text generation with the `generate` UDF.

In [ ]:
gen_t = pxt.create_table('vllm_demo/generation', {'prompt': pxt.String})

gen_t.add_computed_column(
    result=vllm.generate(
        gen_t.prompt,
        model='Qwen/Qwen2.5-0.5B-Instruct',
        model_kwargs={'max_tokens': 100},
    )
)

gen_t.add_computed_column(output=gen_t.result.choices[0].text)

gen_t.insert(
    [
        {'prompt': 'The capital of France is'},
        {'prompt': 'Once upon a time, there was a'},
    ]
)

gen_t.select(gen_t.prompt, gen_t.output).collect()

## Additional Resources

- [Pixeltable Documentation](https://docs.pixeltable.com/)
- [vLLM Documentation](https://docs.vllm.ai/)
- [vLLM GitHub](https://github.com/vllm-project/vllm)